<a href="https://colab.research.google.com/github/saribasmetehan/bank_administrative_assistant/blob/main/Summarization_Mini_instruct_LLM_with_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Labellara özgü verileri .txt yazdırma.

In [ ]:
from datasets import load_dataset
import os

dataset = load_dataset("legacy-datasets/banking77")

label_names = dataset['test'].features['label'].names

test_data = dataset['test']

raw_output_dir = "raw_data"
os.makedirs(raw_output_dir, exist_ok=True)

summary_output_dir = "summaries"
os.makedirs(summary_output_dir, exist_ok=True)

for example in test_data:
    label = label_names[example['label']]

    raw_file_path = os.path.join(raw_output_dir, f"{label}.txt")

    with open(raw_file_path, "a", encoding="utf-8") as f:
        text = example['text'].replace("\n", " ")
        f.write(text + " ")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

# Summarization

In [ ]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=device)

def summarize_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    summary = summarizer(text, max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    return summary[0]['summary_text']

summary_output_file = os.path.join(summary_output_dir, "summarized_data.txt")
with open(summary_output_file, "w", encoding="utf-8") as f:
    for label in label_names:
        raw_file_path = os.path.join(raw_output_dir, f"{label}.txt")
        summary = summarize_file(raw_file_path)
        f.write(f"Label: {label}\n{summary}\n\n")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
with open("/content/summaries/summarized_data.txt", 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

Label: activate_my_card
How do I activate my new card? How do I renew my card? What is the process of card activation? What can I do after the card is missing? How long does it take to activate a new card and how do I use it?

Label: age_limit
How old do you have to be to open an account? What is the minimum age to open a bank account? How do you go about setting up an account for a newborn baby? How old does one need to be in order to have an account with the bank?

Label: apple_pay_or_google_pay
Top up on Google Pay isn't working on Apple Watch. American Express is experiencing a problem with Apple Pay with top up, so he can't top up his account with his American Express in Apple Pay. He wants to get Google Pay and top up app for his watch.

Label: atm_support
There is a list of ATMs that accept the Mastercard card. The card can be used to withdraw money from ATMs. There are ATMs where you can't withdraw money, but you can use the card to get cash anywhere.

Label: automatic_top_up
W

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

summarized_file_path = os.path.join(summary_output_dir, "summarized_data.txt")
with open(summarized_file_path, "r", encoding="utf-8") as f:
    summarized_data = f.read()

results = []
for section in summarized_data.split("\n\n"):
    if section.strip():
        lines = section.strip().split("\n")
        label = lines[0].replace("Label: ", "").strip()
        summary = " ".join(lines[1:])

        messages = [
            {"role": "system", "content": "You are an analyst who examines customer complaints of a bank. Evaluate the summary of complaints of many customers and provide insights and potential solutions."},
            {"role": "user", "content": f"Here is the summary of complaints related to the label '{label}':\n\n{summary}"},
        ]

        output = pipe(messages, **generation_args)
        generated_text = output[0]["generated_text"]

        results.append(f"Label: {label}\nGenerated Output:\n{generated_text}\n")

output_file_path = os.path.join(summary_output_dir, "phi3_model_outputs.txt")
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("\n\n".join(results))


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


In [ ]:
with open("/content/summaries/phi3_model_outputs.txt", 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

Label: activate_my_card
Generated Output:
 Based on the summary of complaints related to the label 'activate_my_card', it appears that customers are facing issues with understanding the process of card activation, renewal, and what to do if their card goes missing. Additionally, there seems to be confusion about the time it takes to activate a new card and how to use it.

To address these concerns, the bank can consider the following solutions:

1. Clear and concise instructions: The bank should provide clear and easy-to-understand instructions on how to activate and renew cards. This can be done through various channels, such as the bank's website, mobile app, and customer service representatives.

2. Step-by-step guides: Create step-by-step guides for card activation and renewal processes, which can be easily accessed by customers. These guides can be shared on the bank's website, social media platforms, and through email newsletters.

3. Video tutorials: The bank can create video tu